In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_parquet('options_data/OptionMetrics.parquet', engine='pyarrow')  # or engine='fastparquet'
print(df)


          secid        date             symbol  symbol_flag      exdate  \
0        101121  2022-08-31  AMD 220902C100000            1  2022-09-02   
1        101121  2022-08-31  AMD 220902C101000            1  2022-09-02   
2        101121  2022-08-31  AMD 220902C102000            1  2022-09-02   
3        101121  2022-08-31  AMD 220902C103000            1  2022-09-02   
4        101121  2022-08-31  AMD 220902C104000            1  2022-09-02   
...         ...         ...                ...          ...         ...   
6541603  213572  2023-08-31    ZM 230929P79000            1  2023-09-29   
6541604  213572  2023-08-31    ZM 230929P80000            1  2023-09-29   
6541605  213572  2023-08-31    ZM 230929P85000            1  2023-09-29   
6541606  213572  2023-08-31    ZM 230929P90000            1  2023-09-29   
6541607  213572  2023-08-31    ZM 230929P95000            1  2023-09-29   

          last_date cp_flag  strike_price  best_bid  best_offer  ...     sic  \
0        2022-08-31

In [3]:
df.sample(10).head(10)

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag
5367547,110013,2023-01-04,XLI 230120P75000,1,2023-01-20,2022-12-30,P,75000,0.00,0.03,...,NaN,0,1,None,%,None,SSGA ACTIVE TRUST,None,A,None
4477347,109820,2023-01-31,SPY 230201P428000,1,2023-02-01,None,P,428000,21.66,21.82,...,NaN,0,1,None,%,None,SSGA ACTIVE TRUST,None,A,None
2772413,103823,2023-01-05,DIA 230203P349000,1,2023-02-03,None,P,349000,19.60,21.25,...,NaN,0,1,None,%,None,SSGA ACTIVE TRUST,None,A,None
6474713,208181,2023-08-17,XLRE 230915P47000,1,2023-09-15,None,P,47000,10.70,11.00,...,NaN,0,1,None,%,None,SSGA ACTIVE TRUST,None,A,None
6285615,126681,2023-04-19,USO 230512P62000,1,2023-05-12,2023-04-19,P,62000,0.34,0.46,...,NaN,0,1,None,%,426,UNITED STATES COMMODITY FUND,None,A,None
961195,108161,2022-11-03,NKE 221118C65000,1,2022-11-18,2022-10-14,C,65000,25.35,25.75,...,3021.0,0,1,None,0,None,"NIKE, INC.",None,A,None
5661968,111953,2023-07-25,WFC 230804P45500,1,2023-08-04,2023-07-25,P,45500,0.91,0.93,...,6021.0,0,1,None,0,None,WELLS FARGO & CO.,None,A,None
4704831,109820,2023-05-19,SPY 230602P417000,1,2023-06-02,2023-05-19,P,417000,3.16,3.18,...,NaN,0,1,None,%,None,SSGA ACTIVE TRUST,None,A,None
1779921,110015,2022-12-06,XLU 221216P55000,1,2022-12-16,2022-12-05,P,55000,0.00,0.04,...,NaN,0,1,None,%,None,SSGA ACTIVE TRUST,None,A,None
2367772,101121,2023-06-30,AMD 230714P107000,1,2023-07-14,2023-06-30,P,107000,0.84,0.86,...,3674.0,0,5,None,0,None,ADVANCED MICRO DEVICES INC.,None,A,None


In [4]:
# split symbol after space
df["ticker"] = df["symbol"].str.split(" ").str[0]

In [5]:
unq_tickers = df["ticker"].unique()

In [2]:
tikcers_of_interest = [
    "JPM",
    "BAC",
    "WFC",
    "PFE",
    "MRK",
    "HD",
    "MCD",
    "NKE",
    "XOM",
    "CVX",
    "COP",
    "AMD",
    "ZM",
    "GME",
    "AMC",
    "BB",
    "NOK",]

In [3]:
import json
with open('config.json', 'r') as f:
    CONFIG = json.load(f)

In [4]:
loaded_tickers = CONFIG["tickers"]["assets"]["s&p 500"]

In [5]:
[t in loaded_tickers for t in tikcers_of_interest]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [6]:
files = os.listdir("options_data")
data = pd.DataFrame()
for file in files:
    if file.endswith(".csv"):
        ticker = file.split(".csv")[0]
        sub_df = pd.read_csv(f"options_data/{file}", index_col=0, parse_dates=True)
        sub_df.rename(columns={"Close": f"{ticker}"}, inplace=True)
        data = data.join(sub_df, how='right')

In [7]:
data_rolling_mean = data.rolling(window=100).mean()
data_rolling_std = data.rolling(window=100).std()
data_normalized = (data - data_rolling_mean) / data_rolling_std

In [8]:
data_normalized.describe()

,MRK,GME,WFC,COP,NOK,CVX,AMD,BB,HD,AMC,MCD,PFE,XOM,JPM,ZM,BAC,NKE
count,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,5938.000000,5938.000000
mean,0.376884,0.041396,0.257975,0.469708,-0.302760,0.120644,0.537678,-0.283871,0.411019,-0.522073,0.453829,-0.036876,0.274819,0.428171,-0.241294,0.293178,0.488231
std,1.274555,1.402288,1.420857,1.334596,1.289573,1.412225,1.385513,1.328937,1.339858,1.287681,1.376254,1.327946,1.385193,1.418816,1.399296,1.367693,1.340901
min,-3.540493,-2.622178,-4.256479,-4.780495,-5.645634,-4.455461,-2.397811,-3.684279,-5.107214,-4.510963,-4.623751,-4.082270,-4.076770,-4.928942,-2.802208,-4.734303,-4.855785
25%,-0.511990,-1.076913,-0.910754,-0.341815,-1.086093,-0.990493,-0.615264,-1.132999,-0.681867,-1.327070,-0.353254,-1.076870,-0.704179,-0.648308,-1.345939,-0.808023,-0.551070
50%,0.298693,0.044179,0.246819,0.573559,-0.327254,0.231166,0.670733,-0.388919,0.654966,-0.736829,0.592055,-0.028912,0.455180,0.634542,-0.638059,0.487912,0.710253
75%,1.360671,0.884314,1.503213,1.508509,0.528351,1.235153,1.618518,0.504163,1.387206,0.187189,1.439568,0.861831,1.329440,1.507653,0.760065,1.385054,1.537350
max,3.816608,8.810640,3.739016,3.704805,7.143984,3.625950,4.333431,5.772725,3.541780,7.936634,3.379656,4.229917,3.203117,4.979608,4.210025,3.542529,4.389607


In [9]:
data.describe()

,MRK,GME,WFC,COP,NOK,CVX,AMD,BB,HD,AMC,MCD,PFE,XOM,JPM,ZM,BAC,NKE
count,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,6037.000000,6037.000000
mean,78.694951,19.969043,38.419441,68.476788,4.200195,111.351535,81.468589,6.571216,264.179582,119.476296,219.540708,34.864068,66.593502,119.738149,172.126592,20.905983,40.225708
std,15.359115,17.425909,7.910923,29.954994,0.690659,33.328259,30.935940,2.589732,50.644091,122.000301,36.593228,6.782365,25.879473,22.262502,127.422227,9.453849,41.129633
min,54.814095,0.700000,19.162121,18.966831,2.205954,43.919682,26.240000,2.900000,135.781647,6.070000,122.877396,22.043989,25.031292,68.480957,59.290001,2.486533,2.525942
25%,67.264387,1.528750,36.347363,45.301328,3.693162,87.366423,55.332500,4.720000,223.689713,45.675000,189.207668,29.327118,48.553999,100.596191,72.470001,12.677497,8.354326
50%,70.608910,19.340000,40.509321,53.916359,4.250358,96.688885,83.705002,5.690000,275.852661,74.500000,219.447609,33.081963,56.566532,121.530079,103.895000,20.571306,20.146955
75%,91.974785,32.120001,43.333157,98.235676,4.648911,146.120426,103.677498,8.222500,299.055740,136.550949,248.785809,40.361297,95.776323,139.014217,270.229988,28.212921,59.478642
max,112.877464,86.877502,54.757721,125.342117,5.970660,173.227661,161.910004,25.100000,385.532440,625.500000,289.494507,53.479515,115.181786,165.473572,568.340027,45.637875,170.152176


In [14]:
data_rolling_mean.describe()

,MRK,GME,WFC,COP,NOK,CVX,AMD,BB,HD,AMC,MCD,PFE,XOM,JPM,ZM,BAC,NKE
count,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,5938.000000,5938.000000
mean,78.302355,21.016612,38.166470,67.276775,4.220773,110.598828,82.133050,6.615798,266.241074,124.675283,219.181091,35.333518,65.384015,119.564917,180.944367,20.910836,39.986080
std,14.525457,16.190477,7.365728,28.642970,0.580541,32.707732,27.027954,2.247151,45.003279,109.934582,34.130115,6.406340,25.331089,19.884389,123.379678,9.313895,40.688804
min,64.485731,1.076500,21.942364,30.974206,3.286871,67.024608,30.220300,4.029000,181.115441,11.718900,163.188484,27.054660,31.603076,82.831940,66.948450,4.973765,3.605687
25%,67.470563,1.418575,36.024616,43.099157,3.760696,83.443710,68.081100,4.801900,236.711311,48.605000,186.059527,29.254056,45.501692,103.614260,75.046900,12.927082,8.279815
50%,70.528107,21.517100,40.629174,54.620489,4.204256,94.907041,84.737600,5.824500,279.158204,76.175000,221.418232,33.550303,54.940981,120.445005,110.080400,20.583952,20.602109
75%,86.971347,32.194000,42.413263,97.804547,4.580510,148.297107,103.052800,8.393700,295.744732,158.390001,244.445231,42.318594,91.149997,139.094526,317.780499,28.318885,56.432598
max,106.611102,48.929125,48.002078,114.137512,5.322638,160.797465,131.407400,11.103600,347.514687,432.065999,280.020133,46.495138,104.506362,149.148207,431.135550,42.568509,157.512191


In [18]:
import numpy as np

In [23]:
log_returns = np.log(data["JPM"]/data["JPM"].shift(1)).dropna()
returns = data["JPM"].pct_change().dropna()

In [28]:
market_data = pd.read_csv("data/market_data/market_data.csv", index_col=0, parse_dates=True)

In [30]:
rf = market_data["^IRX"]

In [31]:
rf.describe()

count    3873.000000
mean        1.050948
std         1.518008
min        -0.105000
25%         0.050000
50%         0.160000
75%         1.683000
max         5.348000
Name: ^IRX, dtype: float64